In [1]:
# -*- coding: utf-8 -*-
import re
import json
import zipfile
from pathlib import Path

import pandas as pd

# ==== 경로 설정 ====
TL_ZIP = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\TL.zip")  # TL.zip 실제 위치로 수정
FOOD_CSV = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\food_ingredients_integration1.csv")  # CSV 위치로 수정
OUT_CSV = TL_ZIP.with_name("image_food_label_join.csv")  # 결과 파일

# ==== 1) 파일 이름에서 음식명 추출 함수 ====
def extract_food_name(file_name: str) -> str:
    """
    예: A_13_A13001_가자미구이_30_09.jpg  ->  '가자미구이'
    """
    stem = file_name.rsplit(".", 1)[0]  # 확장자 제거
    parts = stem.split("_")
    for p in parts:
        # 한글이 들어있는 토큰만 골라오기
        if re.search(r"[가-힣]", p):
            return p
    return ""

# ==== 2) 식약처 CSV 불러오기 ====
food_df = pd.read_csv(FOOD_CSV)
food_names = set(food_df["식품명"])

# ==== 3) TL.zip 안의 JSON들 순회하면서 매칭 ====
rows = []

with zipfile.ZipFile(TL_ZIP) as z:
    for name in z.namelist():
        if not name.endswith(".json"):
            continue

        data = json.loads(z.read(name).decode("utf-8"))
        image_info = data["data"]["image_info"]
        image_file = image_info["file_name"]

        # 파일명에서 음식 이름 추출
        food_from_file = extract_food_name(image_file)

        # 1차: 식품명 완전 일치로 찾기
        exact_matches = food_df[food_df["식품명"] == food_from_file]

        # 2차: 없으면 부분 포함으로 후보 찾기 (예: '물회' -> '물회', '물회_오징어', ...)
        if len(exact_matches) == 0 and food_from_file:
            exact_matches = food_df[food_df["식품명"].str.contains(food_from_file, na=False)]

        if len(exact_matches) == 0:
            # 완전 매칭 실패 → 나중에 엑셀에서 수동으로 채우기
            rows.append({
                "json_path": name,
                "image_file": image_file,
                "food_from_filename": food_from_file,  # JSON에서 추출한 메뉴명
                "식품코드": "",
                "식품명(B열)": "",  # 여기 나중에 직접 채워넣기
                "match_type": "not_found",
            })
        else:
            # 매칭된 후보가 여러 개일 수 있어서 전부 한 줄씩 기록
            for _, row in exact_matches.iterrows():
                rows.append({
                    "json_path": name,
                    "image_file": image_file,
                    "food_from_filename": food_from_file,
                    "식품코드": row["식품코드"],
                    "식품명(B열)": row["식품명"],
                    "match_type": "exact" if row["식품명"] == food_from_file else "contains",
                })

# ==== 4) 결과 저장 ====
out_df = pd.DataFrame(rows)
out_df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("완료! 결과 CSV:", OUT_CSV)

C:\Users\EL19\AppData\Local\Temp\ipykernel_13232\3386827956.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  exact_matches = food_df[food_df["식품명"].str.contains(food_from_file, na=False)]


완료! 결과 CSV: C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\image_food_label_join.csv
